In [69]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .appName("test") \
    .getOrCreate()

In [70]:
df= spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

25/06/26 14:12:44 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [71]:
from pyspark.sql.functions import explode, split

In [72]:
words = df.select(explode(split(df.value, " ")).alias("word"))

# groupby sangai applyinpandaswith state use garda

yellae yesari samjhim

groupby le (key1:[value1,value2,value3],key2:[val1,val2,val3]) yesto dinxa

eg if we type hello hi hello hi

lets say letsa say matrai hai yo exactly kasari hunxa tyo chai bhujnu parxa koi sanga sodhae ra

aaba applyinpandaswithstate ma pathauni yellai

yo function ko arguments vane ko key aani tyo key ko list aani state store vanam

so pratyek key ko lagi yo function call hunxa

key to time limit hunxa tyo bhujam

key ko time sakkini bittikai yo function call hunxa with empty list 

so key yedi aayo yo time vanda aagadi nai time lai feri reset garnu parxa so settimeout garni

or aafai logic lekhae ni ta vayo aaba

like if key=="your key"
    
    then remove logic



aaba state store ko function

state store vanda ni tyo key ko state chai aaudo raixa

so we can do 

1.state.hasTimedOut()

2.state.remove()

3.state.exists()

4.state.get()

5.state.update(newState)

6.state.setTimeoutDuration("1 hour")  # yo time reset garauni na garuni your logic



In [73]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

In [74]:
output_schema = StructType([
    StructField("word", StringType()),
    StructField("count", IntegerType())
])

# Define state schema (just count as int)
state_schema = StructType([
    StructField("count", IntegerType())
])

In [75]:
import pandas as pd

In [76]:
def update_state(key,value_iterator,state):
    with open("/opt/keyvale.txt","a") as file:
        file.write(f"1.key is {key} \n")
        file.write(f"2.value is {list(value_iterator)}")
        file.write(f"\n 3.type of iterator is {type(list(value_iterator))}")
    pdf = pd.DataFrame({
        "word": key,
        "count": 1
    })

    # Return an iterator of one dataframe
    yield pdf

In [77]:
stateful_df = words.groupBy("word").applyInPandasWithState(
    update_state,
    outputStructType=output_schema,
    stateStructType=state_schema,
    outputMode="update",
    timeoutConf="ProcessingTimeTimeout"
)


In [78]:
query = stateful_df.writeStream \
    .outputMode("update") \
    .format("console") \
    .start()

query.awaitTermination()

25/06/26 14:13:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6c3148b1-da6e-4617-bc5c-17d4691d007e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/26 14:13:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
                                                                                

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



...

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    1|
|aaditya|    1|
|     hi|    1|
+-------+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 7
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



ERROR:root:KeyboardInterrupt while sending command.              (98 + 8) / 200]
Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.9/dist-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.9/socket.py", line 704, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [79]:
spark.stop()

25/06/26 14:13:45 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] is aborting.
25/06/26 14:13:45 ERROR WriteToDataSourceV2Exec: Data source write support MicroBatchWrite[epoch: 10, writer: ConsoleWriter[numRows=20, truncate=true]] aborted.
25/06/26 14:13:45 ERROR MicroBatchExecution: Query [id = 3c6026cc-bf21-495e-9166-3e10bd7ed433, runId = 26292219-3b94-48aa-a566-923d07044136] terminated with error
org.apache.spark.SparkException: Job 10 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1301)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1299)
	at scala.collection.mutable.HashSet$Node.foreach(HashSet.scala:450)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:376)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAG